# 推荐模型相关
## 召回

- 介绍双塔模型
- 双塔模型的输出，用双塔embedding做内积+sigmoid和求余弦相似度+sigmoid的区别
- 双塔模型一般怎么做特征
- 双塔模型为什么不直接把两个塔合起来输入一个DNN

## 排序

- 为什么CTR中目前普遍使用深度学习模型替换树模型
- 为什么要有wide层、FM层，deep层不也有记忆能力吗
- DeepFM与wide&deep的介绍与对比
- 对DeepFM进行优化，有哪些思路
- DeepFM如果过拟合和欠拟合分别如何处理
- 介绍除了FM之外的特征交叉的模型
- 介绍DIN模型，适合的场景
- DIN中如何计算attention
- transformer与DIN的区别和联系
- 介绍下listwise排序模型LambdaRank

## 参考解析

## 召回

- 介绍双塔模型
  - 双塔模型最大的特点就是**「user和item是独立的两个子网络」**，左侧是用户塔，右侧是item塔，这两个塔的参数不共享；
  - 输入层：**「[User特征]」**主要包括和用户相关的特征：用户id、手机系统、地域、年龄、历史行为序列等，**「Item特征」**主要包括和Item相关的特征：ItemId、Item类别、Item来源等；
  - 表示层：User特征和Item特征分别输入到特征提取网络（比如说DNN等）得到User Embedding和Item Embedding。之后我们可以计算这两个Embedding之间的余弦距离。**「用户点击过的Item其距离更近，用户没有点击过或者讨厌的Item其距离更远」**。之后利用算得的loss来更新模型的参数。
  - 匹配层：拿用户向量去FAISS中和Item向量进行相似度计算，并返回距离最近的Top K个Item作为个性化的召回结果。

- 双塔模型的输出，用双塔embedding做内积+sigmoid和求余弦相似度+sigmoid的区别
- 双塔模型一般怎么做特征
  - 每个塔各自构建user与item embedding，**[User特征]」**主要包括和用户相关的特征：用户id、手机系统、地域、年龄、历史行为序列等，上下文特征（Context feature）可以放入用户侧塔，**「Item特征」**主要包括和Item相关的特征：ItemId、Item类别、Item来源等；

- 双塔模型为什么不直接把两个塔合起来输入一个DNN
  - 性能：减少线上运算速度，item塔可以提前训练，线上只需要user的embedding和相似度计算；


## 排序

- 为什么CTR中目前普遍使用深度学习模型替换树模型
  - 强大的表达能力，能够挖掘更深层次数据模式；
  - 支持更丰富的特征形式；
  - 模型结构非常灵活，能够根据实际应用场景进行调整
- 为什么要有wide层、FM层，deep层不也有记忆能力吗
  - wide层记忆能力更强，因为它结构简单，原始数据能够直接影响推荐结果，能够学习到数据中的简单规则。我理解的是深度网络的优势的容易学到隐形特征交叉，虽然DNN的万能近似定理能够让网络拟合任何一个函数，但是也有研究发现实际情况下DNN对于各个特征的学习是不充分的。特征最重要的是先验知识，如果将特征加入DNN底层，层层向上传递，到达顶层信息留下就不多了，特别是在推荐系统特征都被embedding后。所以只有足够浅的维度才能让网络学习到重要特征。
- DeepFM与wide&deep的介绍与对比
  - Wide&Deep模型同时考虑了记忆能力和泛化能力，但Wide部分需要人工参与特征工程；DeepFM对Wide&Deep模型的改进之处在于用FM替换了原来的Wide部分,加强了浅层网络部分特征组合的能力。
  - DeepFM的动机非常直观，既希望考虑高/低阶的feature interaction，又想省去额外的特征工程。使用FM取代Wide的LR部分是一个可行的做法，当然这里LR可以基于先验构造更高阶的组合特征，而FM只考虑二阶，DeepFM中的FM层和隐藏层共享输入，这种共享输入使得DeepFM可以同时从原始特征中学习低阶特征交互和高阶特征交互,完全不需要执行特征工程。
- 对DeepFM进行优化，有哪些思路
  - 比较开放的题目，可以把一些其他模型的创新添加进去，比如DCN的更高阶交叉、DIN的序列融入DeepFM。
- DeepFM如果过拟合和欠拟合分别如何处理
  - 结合DNN的过拟合处理思路讲；
- 介绍除了FM之外的特征交叉的模型
  - FNN：有高阶bit-wise特征交叉，每个特征都使用了与训练的FM模型，训练开销更低。
  - DeepFM：是一种可以从原始特征中抽取到各种复杂度特征的端到端模型，没有人工[特征工程](https://www.zhihu.com/search?q=特征工程&search_source=Entity&hybrid_search_source=Entity&hybrid_search_extra={"sourceType"%3A"article"%2C"sourceId"%3A269730650})的困扰，DeepFM模型包含FM和DNN两部分，FM模型可以抽取low-order特征，DNN可以抽取high-order特征。无需类似Wide&Deep模型人工特征工程。
  - DCN：可以任意组合特征，而且不增加网络参数.Cross的目的是以一种显示、可控且高效的方式，自动构造有限高阶交叉特征。
- 介绍DIN模型，适合的场景
  - 如何刻画用户兴趣的广泛性，是推荐系统比较大的一个难点，用户历史行为序列建模的研究经历了从Pooling、RNN到attention、capsule再到transformer的顺序
  - 在DIN之前，业界处理序列特征，普遍是在embedding之后进行pooling，这种做法将序列特征的每个item看作是相等的权重，举个例子：用户历史购买过9件衣服和1个鼠标，本次候选商品是键盘，但是因为用户历史行为被衣服充斥着，历史行为pooling后的embedding非常偏向衣服这个子空间，而历史购买键盘后再买鼠标显然应该赋予更大的权重。通过pooling的做法就会导致历史行为的兴趣被平滑了，学成一个四不像的没法体现用户广泛兴趣的向量。
  - DIN模型提出的动机是利用target attention的方法，进行加权pooling，它为历史行为的物品和当前推荐物品计算一个attention score，然后加权pooling，这样的方法更能体现用户兴趣多样性。
  - DIN模型，增加了注意力机制，模型的创新点或者解决的问题就是使用了注意力机制来对用户的兴趣动态模拟， 而这个模拟过程存在的前提就是用户之前有大量的历史行为了，这样我们在预测某个商品广告用户是否点击的时候，就可以参考他之前购买过或者查看过的商品，这样就能猜测出用户的大致兴趣来，这样我们的推荐才能做的更加到位，所以这个模型的使用场景是**非常注重用户的历史行为特征（历史购买过的商品或者类别信息）**
- transformer与DIN的区别和联系
- 介绍下listwise排序模型LambdaRank
  - Listwise方法是直接优化排序列表，输入为单条样本为一个**文档排列**。通过构造合适的度量函数衡量当前文档排序和最优排序差值，优化度量函数得到排序模型。